In [ ]:
require 'image'
require 'lfs'

In [ ]:
function extract_snippets(fname, snippets, index)
    local left = image.load('data/kitti_stereo/training/image_0/' .. fname, 1, 'byte');
    local right = image.load('data/kitti_stereo/training/image_1/' .. fname, 1, 'byte');
    local disp = image.load('data/kitti_stereo/training/disp_noc/' .. fname, 1, 'float');
    disp = disp:mul(65535/256)

    local count = 0
    for y = 9, left:size(2) - 9 do
        for x = 9, left:size(3) - 9 do
            if disp[1][y][x] > 0 and x - disp[1][y][x] > 9 then
                local left_snip = left:narrow(2, y-4, 9):narrow(3, x-4, 9)
                local x2 = x - disp[1][y][x]
                x2 = math.floor(x2+0.5)
                local right_snip = right:narrow(2, y-4, 9):narrow(3, x2-4, 9)
                count = count + 1
                if snippets ~= nil then
                    snippets[1][index+count-1]:copy(left_snip[1])
                    snippets[2][index+count-1]:copy(right_snip[1])
                end
            end
        end
    end
    collectgarbage()
    return count
end

In [ ]:
left = image.load('data/kitti_stereo/training/image_0/000127_10.png');
right = image.load('data/kitti_stereo/training/image_1/000127_10.png');
disp = image.load('data/kitti_stereo/training/disp_noc/000127_10.png', 1, 'float');
disp = disp:mul(65535/256)
print(string.format('Disp:max: %s', disp:max()))
itorch.image(left);
itorch.image(right);
itorch.image(disp);

apxRight = torch.Tensor(left:size())
apxRight:fill(0)
for y = 1, left:size(2) do
    for x = 1, left:size(3) do
        local idx = math.min(x+disp[1][y][x], left:size(3))
        if idx ~= x then
          apxRight[1][y][idx] = left[1][y][x]
        end
    end
end

itorch.image(apxRight);

--diff = right:type('torch.FloatTensor') - apxRight:type('torch:FloatTensor')
--itorch.image(diff)

--print(left:size(), right:size(), disp:size());

In [ ]:
local path_left = 'data/kitti_stereo/training/disp_noc'
expected_total = 24034491
file_name = string.format('/tmp/snippets.2x%dx9x9.byte_tensor', expected_total)
file_size = 2 * expected_total * 9 * 9
print('Creating a storage ...')
snippets_storage = torch.ByteStorage(file_name, true, file_size)
print('Storage created. Creating a tensor ...')
snippets = torch.ByteTensor(snippets_storage, 1, torch.LongStorage(2, expected_total, 9, 9))
print('snippets: ', snippets:size())
total = 0
for fname in lfs.dir(path_left) do
    if fname ~= "." and fname ~= ".." then
        local f = path_left .. '/' .. fname
        local attr = lfs.attributes(f)
        if attr.mode == "file" and string.match(fname, '[.]png$') then
            local count = extract_snippets(fname, snippets, total+1)
            total = total + count
            print(string.format('%s - %d snippets (%d total)', fname, count, total))
        end
    end
end
print(string.format('Loaded %d snippets, expected %d snippets', total, expected_total))
if expected_total == total then
    print('Expectations and reality match - OK')
else
    print('Unexpected number of snippets - Error')
end